In [1]:
# importing libraries 
import numpy as np
import pandas as pd
import sqlite3


# 1. Python: Download a timeseries of daily deaths per country

In [2]:
#reading the data from the csv file
global_ds = pd.read_csv('C:/Users/Jewin/Desktop/Jovita-DataSciene/DATA/time_series_covid19_deaths_global.csv')

In [3]:
global_ds

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20
0,NaN,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,...,1,1,1,2,4,4,4,4,4,4
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,2,4,5,5,6,8,10,10,11,15
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,17,17,19,21,25,26,29,31,35,44
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,1,1,1,1,3,3,3,6,8,12
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,Turks and Caicos Islands,United Kingdom,21.694000,-71.797900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
252,NaN,MS Zaandam,0.000000,0.000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
253,NaN,Botswana,-22.328500,24.684900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
254,NaN,Burundi,-3.373100,29.918900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#deleting the columns not required for analysis
global_ds=global_ds.drop(columns=['Province/State','Lat','Long'])

In [5]:
global_ds=global_ds.rename(columns={"Country/Region": "country"})

In [6]:
global_ds

,country,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,1,1,1,2,4,4,4,4,4,4
1,Albania,0,0,0,0,0,0,0,0,0,...,2,4,5,5,6,8,10,10,11,15
2,Algeria,0,0,0,0,0,0,0,0,0,...,17,17,19,21,25,26,29,31,35,44
3,Andorra,0,0,0,0,0,0,0,0,0,...,1,1,1,1,3,3,3,6,8,12
4,Angola,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,United Kingdom,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
252,MS Zaandam,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
253,Botswana,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
254,Burundi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
data=global_ds.melt(id_vars=["country"], 
        var_name="Date", 
        value_name="Death_Count")

In [24]:
data

,country,Date,Death_Count
0,Afghanistan,2020-01-22,0
1,Albania,2020-01-22,0
2,Algeria,2020-01-22,0
3,Andorra,2020-01-22,0
4,Angola,2020-01-22,0
...,...,...,...
12595,Burma,2020-03-31,1
12596,MS Zaandam,2020-03-31,0
12597,Botswana,2020-03-31,1
12598,Burundi,2020-03-31,0


In [10]:
data.dtypes

country        object
Date           object
Death_Count     int64
dtype: object

In [11]:
data['Date'] = pd.to_datetime(data['Date'])

In [12]:
data = data.groupby(['country','Date'], as_index=False, sort=False)['Death_Count'].sum()

In [13]:
conn = sqlite3.connect('Covid19.db')
c = conn.cursor()
c.execute('CREATE TABLE deaths_total(country,Date,Death_Count)')
conn.commit()


In [26]:
#c.execute('DROP TABLE deaths_total')

In [14]:
data.to_sql('deaths_total', conn, if_exists='replace', index = False)

In [41]:
daily_deaths=data[['country','Date','Death_Count']]

In [42]:
daily_deaths

,country,Date,Death_Count
0,Afghanistan,2020-01-22,0
1,Albania,2020-01-22,0
2,Algeria,2020-01-22,0
3,Andorra,2020-01-22,0
4,Angola,2020-01-22,0
...,...,...,...
12595,Burma,2020-03-31,1
12596,MS Zaandam,2020-03-31,0
12597,Botswana,2020-03-31,1
12598,Burundi,2020-03-31,0


In [43]:
daily_deaths['Count_Increase'] = ((data['Death_Count']-data.sort_values(by=['Date'], ascending=True)
                       .groupby(['country'])['Death_Count'].shift(1,fill_value=0)))

In [53]:
daily_deaths1[daily_deaths1['country']=='China'].head(20)

,country,Date,Death_Count,Count_Increase
32,China,2020-01-22,17,17
212,China,2020-01-23,18,1
392,China,2020-01-24,26,8
572,China,2020-01-25,42,16
752,China,2020-01-26,56,14
932,China,2020-01-27,82,26
1112,China,2020-01-28,131,49
1292,China,2020-01-29,133,2
1472,China,2020-01-30,171,38
1652,China,2020-01-31,213,42


In [50]:
daily_deaths1=data[['country','Date','Death_Count']]


In [51]:
daily_deaths1['Count_Increase'] = ((daily_deaths1['Death_Count']-daily_deaths1.sort_values(by=['Date'], ascending=True)
                       .groupby(['country'])['Death_Count'].shift(1,fill_value=0)))

In [18]:
addColumn = "ALTER TABLE deaths_total ADD COLUMN Death_Increase interger"
c.execute(addColumn)

In [ ]:
#Working query 
c.execute("update deaths_total as A set Death_Increase = Death_Count-(select max(Death_Count) from deaths_total as B where B.country=A.country and B.Date<A.Date)")

In [19]:

#c.execute("update deaths_total set Death_Increase = (select Death_Count-lag(Death_Count, 1, 0) OVER(PARTITION BY Country ORDER BY Date) from deaths_total as A where deaths_total.country=A.country and deaths_total.Date=A.Date)")
#c.execute("MERGE INTO (SELECT country, Date, ROWID rid, Death_Count FROM deaths_total) A USING (select lag(Death_Count, 1, 0) over(partition by country order by Date) as Prev_Day, ROWID as rid from deaths_total) B ON (A.rid = B.rid) WHEN MATCHED THEN UPDATE SET A.Death_Increase = B.Prev_Day,A.Date = sysdate")

#c.execute("update deaths_total as A set Death_Increase = Death_Count-(select prev_dt from (select lag(Death_Count, 1, 0) OVER(PARTITION BY Country ORDER BY Date) as prev_dt,ROWID as rid from deaths_total B) z where z.rid=A.ROWID)")
c.execute("update deaths_total as A set Death_Increase = A.Death_Count-(select prev_dt from (select lag(B.Death_Count, 1, 0) OVER(PARTITION BY Country ORDER BY Date) as prev_dt,ROWID as rid from deaths_total B) z where z.rid=A.ROWID)")

#c.execute("update deaths_total as A set Death_Increase = Death_Count-(select lag(Death_Count, 1, 0) OVER(PARTITION BY Country ORDER BY Date) from deaths_total)")

In [20]:
c.execute("SELECT * FROM deaths_total")
rows = c.fetchall()
for row in rows:
        print(row)

('Afghanistan', '2020-01-22 00:00:00', 0, 0)
('Albania', '2020-01-22 00:00:00', 0, 0)
('Algeria', '2020-01-22 00:00:00', 0, 0)
('Andorra', '2020-01-22 00:00:00', 0, 0)
('Angola', '2020-01-22 00:00:00', 0, 0)
('Antigua and Barbuda', '2020-01-22 00:00:00', 0, 0)
('Argentina', '2020-01-22 00:00:00', 0, 0)
('Armenia', '2020-01-22 00:00:00', 0, 0)
('Australia', '2020-01-22 00:00:00', 0, 0)
('Austria', '2020-01-22 00:00:00', 0, 0)
('Azerbaijan', '2020-01-22 00:00:00', 0, 0)
('Bahamas', '2020-01-22 00:00:00', 0, 0)
('Bahrain', '2020-01-22 00:00:00', 0, 0)
('Bangladesh', '2020-01-22 00:00:00', 0, 0)
('Barbados', '2020-01-22 00:00:00', 0, 0)
('Belarus', '2020-01-22 00:00:00', 0, 0)
('Belgium', '2020-01-22 00:00:00', 0, 0)
('Benin', '2020-01-22 00:00:00', 0, 0)
('Bhutan', '2020-01-22 00:00:00', 0, 0)
('Bolivia', '2020-01-22 00:00:00', 0, 0)
('Bosnia and Herzegovina', '2020-01-22 00:00:00', 0, 0)
('Brazil', '2020-01-22 00:00:00', 0, 0)
('Brunei', '2020-01-22 00:00:00', 0, 0)
('Bulgaria', '2020-01

('Spain', '2020-01-26 00:00:00', 0, 0)
('Sri Lanka', '2020-01-26 00:00:00', 0, 0)
('Sudan', '2020-01-26 00:00:00', 0, 0)
('Suriname', '2020-01-26 00:00:00', 0, 0)
('Sweden', '2020-01-26 00:00:00', 0, 0)
('Switzerland', '2020-01-26 00:00:00', 0, 0)
('Taiwan*', '2020-01-26 00:00:00', 0, 0)
('Tanzania', '2020-01-26 00:00:00', 0, 0)
('Thailand', '2020-01-26 00:00:00', 0, 0)
('Togo', '2020-01-26 00:00:00', 0, 0)
('Trinidad and Tobago', '2020-01-26 00:00:00', 0, 0)
('Tunisia', '2020-01-26 00:00:00', 0, 0)
('Turkey', '2020-01-26 00:00:00', 0, 0)
('Uganda', '2020-01-26 00:00:00', 0, 0)
('Ukraine', '2020-01-26 00:00:00', 0, 0)
('United Arab Emirates', '2020-01-26 00:00:00', 0, 0)
('United Kingdom', '2020-01-26 00:00:00', 0, 0)
('Uruguay', '2020-01-26 00:00:00', 0, 0)
('US', '2020-01-26 00:00:00', 0, 0)
('Uzbekistan', '2020-01-26 00:00:00', 0, 0)
('Venezuela', '2020-01-26 00:00:00', 0, 0)
('Vietnam', '2020-01-26 00:00:00', 0, 0)
('Zambia', '2020-01-26 00:00:00', 0, 0)
('Zimbabwe', '2020-01-26 00

('Japan', '2020-01-30 00:00:00', 0, 0)
('Jordan', '2020-01-30 00:00:00', 0, 0)
('Kazakhstan', '2020-01-30 00:00:00', 0, 0)
('Kenya', '2020-01-30 00:00:00', 0, 0)
('Korea, South', '2020-01-30 00:00:00', 0, 0)
('Kuwait', '2020-01-30 00:00:00', 0, 0)
('Kyrgyzstan', '2020-01-30 00:00:00', 0, 0)
('Latvia', '2020-01-30 00:00:00', 0, 0)
('Lebanon', '2020-01-30 00:00:00', 0, 0)
('Liberia', '2020-01-30 00:00:00', 0, 0)
('Liechtenstein', '2020-01-30 00:00:00', 0, 0)
('Lithuania', '2020-01-30 00:00:00', 0, 0)
('Luxembourg', '2020-01-30 00:00:00', 0, 0)
('Madagascar', '2020-01-30 00:00:00', 0, 0)
('Malaysia', '2020-01-30 00:00:00', 0, 0)
('Maldives', '2020-01-30 00:00:00', 0, 0)
('Malta', '2020-01-30 00:00:00', 0, 0)
('Mauritania', '2020-01-30 00:00:00', 0, 0)
('Mauritius', '2020-01-30 00:00:00', 0, 0)
('Mexico', '2020-01-30 00:00:00', 0, 0)
('Moldova', '2020-01-30 00:00:00', 0, 0)
('Monaco', '2020-01-30 00:00:00', 0, 0)
('Mongolia', '2020-01-30 00:00:00', 0, 0)
('Montenegro', '2020-01-30 00:00:00

('Dominican Republic', '2020-02-02 00:00:00', 0, 0)
('Ecuador', '2020-02-02 00:00:00', 0, 0)
('Egypt', '2020-02-02 00:00:00', 0, 0)
('El Salvador', '2020-02-02 00:00:00', 0, 0)
('Equatorial Guinea', '2020-02-02 00:00:00', 0, 0)
('Eritrea', '2020-02-02 00:00:00', 0, 0)
('Estonia', '2020-02-02 00:00:00', 0, 0)
('Eswatini', '2020-02-02 00:00:00', 0, 0)
('Ethiopia', '2020-02-02 00:00:00', 0, 0)
('Fiji', '2020-02-02 00:00:00', 0, 0)
('Finland', '2020-02-02 00:00:00', 0, 0)
('France', '2020-02-02 00:00:00', 0, 0)
('Gabon', '2020-02-02 00:00:00', 0, 0)
('Gambia', '2020-02-02 00:00:00', 0, 0)
('Georgia', '2020-02-02 00:00:00', 0, 0)
('Germany', '2020-02-02 00:00:00', 0, 0)
('Ghana', '2020-02-02 00:00:00', 0, 0)
('Greece', '2020-02-02 00:00:00', 0, 0)
('Guatemala', '2020-02-02 00:00:00', 0, 0)
('Guinea', '2020-02-02 00:00:00', 0, 0)
('Guyana', '2020-02-02 00:00:00', 0, 0)
('Haiti', '2020-02-02 00:00:00', 0, 0)
('Holy See', '2020-02-02 00:00:00', 0, 0)
('Honduras', '2020-02-02 00:00:00', 0, 0)
(

('Tanzania', '2020-02-05 00:00:00', 0, 0)
('Thailand', '2020-02-05 00:00:00', 0, 0)
('Togo', '2020-02-05 00:00:00', 0, 0)
('Trinidad and Tobago', '2020-02-05 00:00:00', 0, 0)
('Tunisia', '2020-02-05 00:00:00', 0, 0)
('Turkey', '2020-02-05 00:00:00', 0, 0)
('Uganda', '2020-02-05 00:00:00', 0, 0)
('Ukraine', '2020-02-05 00:00:00', 0, 0)
('United Arab Emirates', '2020-02-05 00:00:00', 0, 0)
('United Kingdom', '2020-02-05 00:00:00', 0, 0)
('Uruguay', '2020-02-05 00:00:00', 0, 0)
('US', '2020-02-05 00:00:00', 0, 0)
('Uzbekistan', '2020-02-05 00:00:00', 0, 0)
('Venezuela', '2020-02-05 00:00:00', 0, 0)
('Vietnam', '2020-02-05 00:00:00', 0, 0)
('Zambia', '2020-02-05 00:00:00', 0, 0)
('Zimbabwe', '2020-02-05 00:00:00', 0, 0)
('Dominica', '2020-02-05 00:00:00', 0, 0)
('Grenada', '2020-02-05 00:00:00', 0, 0)
('Mozambique', '2020-02-05 00:00:00', 0, 0)
('Syria', '2020-02-05 00:00:00', 0, 0)
('Timor-Leste', '2020-02-05 00:00:00', 0, 0)
('Belize', '2020-02-05 00:00:00', 0, 0)
('Laos', '2020-02-05 00

...

('Burkina Faso', '2020-03-14 00:00:00', 0, 0)
('Cabo Verde', '2020-03-14 00:00:00', 0, 0)
('Cambodia', '2020-03-14 00:00:00', 0, 0)
('Cameroon', '2020-03-14 00:00:00', 0, 0)
('Canada', '2020-03-14 00:00:00', 1, 0)
('Central African Republic', '2020-03-14 00:00:00', 0, 0)
('Chad', '2020-03-14 00:00:00', 0, 0)
('Chile', '2020-03-14 00:00:00', 0, 0)
('China', '2020-03-14 00:00:00', 3193, 13)
('Colombia', '2020-03-14 00:00:00', 0, 0)
('Congo (Brazzaville)', '2020-03-14 00:00:00', 0, 0)
('Congo (Kinshasa)', '2020-03-14 00:00:00', 0, 0)
('Costa Rica', '2020-03-14 00:00:00', 0, 0)
("Cote d'Ivoire", '2020-03-14 00:00:00', 0, 0)
('Croatia', '2020-03-14 00:00:00', 0, 0)
('Diamond Princess', '2020-03-14 00:00:00', 7, 0)
('Cuba', '2020-03-14 00:00:00', 0, 0)
('Cyprus', '2020-03-14 00:00:00', 0, 0)
('Czechia', '2020-03-14 00:00:00', 0, 0)
('Denmark', '2020-03-14 00:00:00', 1, 1)
('Djibouti', '2020-03-14 00:00:00', 0, 0)
('Dominican Republic', '2020-03-14 00:00:00', 0, 0)
('Ecuador', '2020-03-14 00:

('Jamaica', '2020-03-16 00:00:00', 0, 0)
('Japan', '2020-03-16 00:00:00', 27, 5)
('Jordan', '2020-03-16 00:00:00', 0, 0)
('Kazakhstan', '2020-03-16 00:00:00', 0, 0)
('Kenya', '2020-03-16 00:00:00', 0, 0)
('Korea, South', '2020-03-16 00:00:00', 75, 0)
('Kuwait', '2020-03-16 00:00:00', 0, 0)
('Kyrgyzstan', '2020-03-16 00:00:00', 0, 0)
('Latvia', '2020-03-16 00:00:00', 0, 0)
('Lebanon', '2020-03-16 00:00:00', 3, 0)
('Liberia', '2020-03-16 00:00:00', 0, 0)
('Liechtenstein', '2020-03-16 00:00:00', 0, 0)
('Lithuania', '2020-03-16 00:00:00', 0, 0)
('Luxembourg', '2020-03-16 00:00:00', 1, 0)
('Madagascar', '2020-03-16 00:00:00', 0, 0)
('Malaysia', '2020-03-16 00:00:00', 0, 0)
('Maldives', '2020-03-16 00:00:00', 0, 0)
('Malta', '2020-03-16 00:00:00', 0, 0)
('Mauritania', '2020-03-16 00:00:00', 0, 0)
('Mauritius', '2020-03-16 00:00:00', 0, 0)
('Mexico', '2020-03-16 00:00:00', 0, 0)
('Moldova', '2020-03-16 00:00:00', 0, 0)
('Monaco', '2020-03-16 00:00:00', 0, 0)
('Mongolia', '2020-03-16 00:00:00'

('Denmark', '2020-03-21 00:00:00', 13, 4)
('Djibouti', '2020-03-21 00:00:00', 0, 0)
('Dominican Republic', '2020-03-21 00:00:00', 2, 0)
('Ecuador', '2020-03-21 00:00:00', 7, 2)
('Egypt', '2020-03-21 00:00:00', 10, 2)
('El Salvador', '2020-03-21 00:00:00', 0, 0)
('Equatorial Guinea', '2020-03-21 00:00:00', 0, 0)
('Eritrea', '2020-03-21 00:00:00', 0, 0)
('Estonia', '2020-03-21 00:00:00', 0, 0)
('Eswatini', '2020-03-21 00:00:00', 0, 0)
('Ethiopia', '2020-03-21 00:00:00', 0, 0)
('Fiji', '2020-03-21 00:00:00', 0, 0)
('Finland', '2020-03-21 00:00:00', 1, 1)
('France', '2020-03-21 00:00:00', 563, 112)
('Gabon', '2020-03-21 00:00:00', 1, 0)
('Gambia', '2020-03-21 00:00:00', 0, 0)
('Georgia', '2020-03-21 00:00:00', 0, 0)
('Germany', '2020-03-21 00:00:00', 84, 17)
('Ghana', '2020-03-21 00:00:00', 1, 1)
('Greece', '2020-03-21 00:00:00', 13, 7)
('Guatemala', '2020-03-21 00:00:00', 1, 0)
('Guinea', '2020-03-21 00:00:00', 0, 0)
('Guyana', '2020-03-21 00:00:00', 1, 0)
('Haiti', '2020-03-21 00:00:00',

('Azerbaijan', '2020-03-25 00:00:00', 2, 1)
('Bahamas', '2020-03-25 00:00:00', 0, 0)
('Bahrain', '2020-03-25 00:00:00', 4, 1)
('Bangladesh', '2020-03-25 00:00:00', 5, 1)
('Barbados', '2020-03-25 00:00:00', 0, 0)
('Belarus', '2020-03-25 00:00:00', 0, 0)
('Belgium', '2020-03-25 00:00:00', 178, 56)
('Benin', '2020-03-25 00:00:00', 0, 0)
('Bhutan', '2020-03-25 00:00:00', 0, 0)
('Bolivia', '2020-03-25 00:00:00', 0, 0)
('Bosnia and Herzegovina', '2020-03-25 00:00:00', 3, 0)
('Brazil', '2020-03-25 00:00:00', 59, 13)
('Brunei', '2020-03-25 00:00:00', 0, 0)
('Bulgaria', '2020-03-25 00:00:00', 3, 0)
('Burkina Faso', '2020-03-25 00:00:00', 4, 0)
('Cabo Verde', '2020-03-25 00:00:00', 1, 0)
('Cambodia', '2020-03-25 00:00:00', 0, 0)
('Cameroon', '2020-03-25 00:00:00', 1, 1)
('Canada', '2020-03-25 00:00:00', 30, 4)
('Central African Republic', '2020-03-25 00:00:00', 0, 0)
('Chad', '2020-03-25 00:00:00', 0, 0)
('Chile', '2020-03-25 00:00:00', 3, 1)
('China', '2020-03-25 00:00:00', 3285, 4)
('Colombia'

('Sierra Leone', '2020-03-29 00:00:00', 0, 0)
('Afghanistan', '2020-03-30 00:00:00', 4, 0)
('Albania', '2020-03-30 00:00:00', 11, 1)
('Algeria', '2020-03-30 00:00:00', 35, 4)
('Andorra', '2020-03-30 00:00:00', 8, 2)
('Angola', '2020-03-30 00:00:00', 2, 0)
('Antigua and Barbuda', '2020-03-30 00:00:00', 0, 0)
('Argentina', '2020-03-30 00:00:00', 23, 4)
('Armenia', '2020-03-30 00:00:00', 3, 0)
('Australia', '2020-03-30 00:00:00', 17, 1)
('Austria', '2020-03-30 00:00:00', 108, 22)
('Azerbaijan', '2020-03-30 00:00:00', 4, 0)
('Bahamas', '2020-03-30 00:00:00', 0, 0)
('Bahrain', '2020-03-30 00:00:00', 4, 0)
('Bangladesh', '2020-03-30 00:00:00', 5, 0)
('Barbados', '2020-03-30 00:00:00', 0, 0)
('Belarus', '2020-03-30 00:00:00', 0, 0)
('Belgium', '2020-03-30 00:00:00', 513, 82)
('Benin', '2020-03-30 00:00:00', 0, 0)
('Bhutan', '2020-03-30 00:00:00', 0, 0)
('Bolivia', '2020-03-30 00:00:00', 4, 3)
('Bosnia and Herzegovina', '2020-03-30 00:00:00', 10, 4)
('Brazil', '2020-03-30 00:00:00', 159, 23)
(

In [21]:
#df = pd.read_sql_query("select * from deaths_total;", conn)
df = pd.read_sql_query("select * from deaths_total as A;", conn)
df

,country,Date,Death_Count,Death_Increase
0,Afghanistan,2020-01-22 00:00:00,0,0
1,Albania,2020-01-22 00:00:00,0,0
2,Algeria,2020-01-22 00:00:00,0,0
3,Andorra,2020-01-22 00:00:00,0,0
4,Angola,2020-01-22 00:00:00,0,0
...,...,...,...,...
12595,Burma,2020-03-31 00:00:00,1,1
12596,MS Zaandam,2020-03-31 00:00:00,0,0
12597,Botswana,2020-03-31 00:00:00,1,1
12598,Burundi,2020-03-31 00:00:00,0,0


In [22]:
test = pd.read_sql_query("select country,Date,Death_Count,Death_Count-lag(Death_Count, 1, 0) OVER(PARTITION BY Country ORDER BY Date) from deaths_total",conn)
test

,country,Date,Death_Count,"Death_Count-lag(Death_Count, 1, 0) OVER(PARTITION BY Country ORDER BY Date)"
0,Afghanistan,2020-01-22 00:00:00,0,0
1,Afghanistan,2020-01-23 00:00:00,0,0
2,Afghanistan,2020-01-24 00:00:00,0,0
3,Afghanistan,2020-01-25 00:00:00,0,0
4,Afghanistan,2020-01-26 00:00:00,0,0
...,...,...,...,...
12595,Zimbabwe,2020-03-27 00:00:00,1,0
12596,Zimbabwe,2020-03-28 00:00:00,1,0
12597,Zimbabwe,2020-03-29 00:00:00,1,0
12598,Zimbabwe,2020-03-30 00:00:00,1,0


In [24]:
df[df['country']=='China'].tail(20)

,country,Date,Death_Count,Death_Increase
9032,China,2020-03-12 00:00:00,3172,11
9212,China,2020-03-13 00:00:00,3180,8
9392,China,2020-03-14 00:00:00,3193,13
9572,China,2020-03-15 00:00:00,3203,10
9752,China,2020-03-16 00:00:00,3217,14
9932,China,2020-03-17 00:00:00,3230,13
10112,China,2020-03-18 00:00:00,3241,11
10292,China,2020-03-19 00:00:00,3249,8
10472,China,2020-03-20 00:00:00,3253,4
10652,China,2020-03-21 00:00:00,3259,6


In [24]:
test[test['country']=='China'].tail(20)

,country,Date,Death_Count,"Death_Count-lag(Death_Count, 1, 0) OVER(PARTITION BY Country ORDER BY Date)"
2570,China,3/2/20,2914,-335
2571,China,3/20/20,3253,339
2572,China,3/21/20,3259,6
2573,China,3/22/20,3274,15
2574,China,3/23/20,3274,0
2575,China,3/24/20,3281,7
2576,China,3/25/20,3285,4
2577,China,3/26/20,3291,6
2578,China,3/27/20,3296,5
2579,China,3/28/20,3299,3


In [25]:
conn.commit()

In [54]:
c.execute("CREATE TABLE deaths_change_sql AS SELECT country,Date,Death_Count,Death_Count-lag(Death_Count, 1, 0) OVER(PARTITION BY Country ORDER BY Date) from deaths_total")
conn.commit()

In [55]:
test2 = pd.read_sql_query("select * from deaths_change_sql",conn)
test2

,country,Date,Death_Count,"Death_Count-lag(Death_Count, 1, 0) OVER(PARTITION BY Country ORDER BY Date)"
0,Afghanistan,2020-01-22 00:00:00,0,0
1,Afghanistan,2020-01-23 00:00:00,0,0
2,Afghanistan,2020-01-24 00:00:00,0,0
3,Afghanistan,2020-01-25 00:00:00,0,0
4,Afghanistan,2020-01-26 00:00:00,0,0
...,...,...,...,...
12595,Zimbabwe,2020-03-27 00:00:00,1,0
12596,Zimbabwe,2020-03-28 00:00:00,1,0
12597,Zimbabwe,2020-03-29 00:00:00,1,0
12598,Zimbabwe,2020-03-30 00:00:00,1,0


In [57]:
test2[test2['country']=='China'].head(20)

,country,Date,Death_Count,"Death_Count-lag(Death_Count, 1, 0) OVER(PARTITION BY Country ORDER BY Date)"
2520,China,2020-01-22 00:00:00,17,17
2521,China,2020-01-23 00:00:00,18,1
2522,China,2020-01-24 00:00:00,26,8
2523,China,2020-01-25 00:00:00,42,16
2524,China,2020-01-26 00:00:00,56,14
2525,China,2020-01-27 00:00:00,82,26
2526,China,2020-01-28 00:00:00,131,49
2527,China,2020-01-29 00:00:00,133,2
2528,China,2020-01-30 00:00:00,171,38
2529,China,2020-01-31 00:00:00,213,42
